Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import pandas as pd
import numpy as np
import ast
import mysql.connector

In [2]:
df_ataques = pd.read_csv('../Ficheros/attack_limpieza5.csv', index_col= 0)
df_clima = pd.read_csv('../Ficheros/ETL1.csv', index_col= 0)

In [3]:
class Carga:
    
    # constructor
    def __init__(self, nombre_bbdd, contraseña):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
        
        
    # método para crear la BBDD  
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
            
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    # Método para ejecutar la inserción de datos en clima     
    def insertar_clima(self, dataframe):
        
        tabla_clima = """ CREATE TABLE IF NOT EXISTS clima (
            idclima INT NOT NULL AUTO_INCREMENT,
            timepoint INT NOT NULL,
            cloudcover INT NOT NULL,
            highcloud INT NOT NULL,
            midcloud  INT NOT NULL,
            lowcloud  INT NOT NULL,
            rh_profile VARCHAR(1000) NOT NULL,
            wind_profile VARCHAR(1000) NOT NULL,
            temp2m INT NOT NULL,
            lifted_index INT NOT NULL,
            rh2m INT NOT NULL,
            msl_pressure INT NOT NULL, 
            prec_type VARCHAR(255) NOT NULL,
            prec_amount INT NOT NULL,
            snow_depth INT NOT NULL,
            wind10m_direction INT NOT NULL,
            wind10m_speed INT NOT NULL,
            latitud FLOAT NOT NULL,
            longitud FLOAT NOT NULL,
            paises VARCHAR(255) NOT NULL,
            PRIMARY KEY (idclima))""" 
        
        self.crear_insertar_tabla(tabla_clima)
        
        for indice, fila in dataframe.iterrows(): # itreamos por el dataframe
        
            insert_clima = f"""            
                    INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile, wind_profile, temp2m, lifted_index, rh2m, 
                    msl_pressure, prec_type, prec_amount, snow_depth, wind10m_direction, wind10m_speed, latitud, longitud, paises) 
                    VALUES ( "{fila["timepoint"]}","{fila['cloudcover']}", "{fila['highcloud']}", "{fila['midcloud']}", "{fila['lowcloud']}",
                    "{fila['rh_profile']}","{fila['wind_profile']}","{fila['temp2m']}","{fila['lifted_index']}","{fila['rh2m']}","{fila['msl_pressure']}"
                    ,"{fila['prec_type']}","{fila['prec_amount']}","{fila['snow_depth']}","{fila['wind10m.direction']}"
                    ,"{fila['wind10m.speed']}","{fila['latitud']}","{fila['longitud']}","{fila['paises']}");
                    """

            self.crear_insertar_tabla(insert_clima)
                        
    
    # Método para ejecutar la inserción de datos en ataques 
    def insertar_ataques(self, dataframe):
        
        tabla_ataques = """ CREATE TABLE IF NOT EXISTS ataques (
            case_number VARCHAR(255) NOT NULL,
            year INT NOT NULL,
            country VARCHAR(255),
            activity VARCHAR(255),
            species_ VARCHAR(255),
            fatal_ VARCHAR(255),
            sex2 VARCHAR(255) NOT NULL,  
            nuevas_edades FLOAT NOT NULL,
            PRIMARY KEY (case_number))"""
            
        self.crear_insertar_tabla(tabla_ataques)
        
        for indice, fila in dataframe.iterrows(): # itreamos por el dataframe
        
            insert_ataques = f"""            
                    INSERT INTO ataques (case_number, year, country, activity, species_, fatal_, sex2, nuevas_edades)
                    VALUES ( "{fila["case_number"]}","{fila['year']}","{fila['country']}","{fila['activity']}","{fila['species_']}","{fila['fatal_']}","{fila['sex2']}","{fila['nuevas_edades']}");
                    """
                    
            self.crear_insertar_tabla(insert_ataques) 

In [4]:
instancia = Carga("tiburones", "AlumnaAdalab")

In [5]:
instancia.crear_bbdd()

Conexión realizada con éxito


In [6]:
instancia.insertar_clima(df_clima)

In [7]:
instancia.insertar_ataques(df_ataques)